## Test 2

```ps1
Pwsh: 7.2.5

# variable config module names
   Benchmark:  @~3696 Mhz

Technique                       Time            RelativeSpeed Throughput
---------                       ----            ------------- ----------
Gcm | ? CommandType -eq "alias" 00:00:00.624909 5.13x         160.02/s
2Step $All.Where{}              00:00:00.494311 4.06x         202.3/s
Inline $All.Where{}             00:00:00.427924 3.51x         233.69/s
Gcm -CommandType Alias          00:00:00.121839 1x            820.75/s

# hardcoded module names

Gcm | ? CommandType -eq "alias" 00:00:00.582083 5.76x         171.8/s
2Step $All.Where{}              00:00:00.557338 5.51x         179.42/s
Inline $All.Where{}             00:00:00.462835 4.58x         216.06/s
Gcm -CommandType Alias          00:00:00.101089 1x            989.22/s
```

## Test 1

```ps1
Pwsh: 7.2.5
# Compare GCM itself, without filtering
   Benchmark:  @~3696 Mhz

Technique                   Time            RelativeSpeed Throughput
---------                   ----            ------------- ----------
Gcm * (explicit)            00:00:47.655236 456.64x       2.1/s
Gcm * (implicit)            00:00:29.668240 284.28x       3.37/s
Gcm -m                      00:00:00.154693 1.48x         646.44/s
Gcm -m x -CommandType Alias 00:00:00.104361 1x            958.21/s
```

In [ ]:
get-date | Join-String -op 'Last ran: ' -format '{0:u}'
$PSVersionTable.PSVersion | Join-String -op 'Pwsh: '
$jstrModule = @{ 'sep' = ', ' ; 'Prop' = { $_.Name, $_.Version -join ': ' } }

if($PSVersionTable.PSEdition -eq 'Desktop') {
    Import-Module -pass 'Benchpress'
    | Join-string @jStrModule | ft
} else {
    # (currently) the fix for core isn't published, so clone it
    Remove-module BenchPress -ea ignore
    Import-Module -passthru (gi 'G:\2022-github\Pwsh\StartAutomating👨\Benchpress\BenchPress.psd1')
    | Join-string @jStrModule | ft
}

Last ran: 2022-08-07 18:03:02Z
Pwsh: 7.2.5
BenchPress: 1.3.2


In [ ]:
$PSVersionTable.PSVersion | Join-String -op 'Pwsh: '
$Config = @{ Names = 'dev.nin' } 
h1 'variable config module names'
Measure-Benchmark -Technique @{
    # 'Filtering by the Source Command' = {
    'Gcm -CommandType Alias' = {
        Get-Command -m $Config.Names -CommandType Alias
    }
    'Gcm | ? CommandType -eq "alias"' = {
        Get-Command -m $Config.Names
        | ? CommandType -eq 'alias'        
    }
    '2Step $All.Where{}' = {
        $allCmd = Get-Command -m $Config.Names
        $allCmd.Where({$_.CommandType -eq 'alias'})
    }
    'Inline $All.Where{} ' = {
        @(Get-Command -m $Config.Names).Where({$_.CommandType -eq 'alias'})
    }
} | sort Time -desc

h1 'hardcoded module names'
Measure-Benchmark -Technique @{
    # 'Filtering by the Source Command' = {
    'Gcm -CommandType Alias' = {
        Get-Command -m 'Dev.nin' -CommandType Alias
    }
    'Gcm | ? CommandType -eq "alias"' = {
        Get-Command -m 'Dev.nin'
        | ? CommandType -eq 'alias'        
    }
    '2Step $All.Where{}' = {
        $allCmd = Get-Command -m 'Dev.nin'
        $allCmd.Where({$_.CommandType -eq 'alias'})
    }
    'Inline $All.Where{} ' = {
        @(Get-Command -m 'Dev.nin').Where({$_.CommandType -eq 'alias'})
    }
} | sort Time -desc

Pwsh: 7.2.5

# variable config module names




   Benchmark:  @~3696 Mhz

Technique                       Time            RelativeSpeed Throughput
---------                       ----            ------------- ----------
2Step $All.Where{}              00:00:00.419902 4.61x         238.15/s
Gcm | ? CommandType -eq "alias" 00:00:00.404342 4.44x         247.31/s
Inline $All.Where{}             00:00:00.323350 3.55x         309.26/s
Gcm -CommandType Alias          00:00:00.091117 1x            1097.49/s

# hardcoded module names

Gcm | ? CommandType -eq "alias" 00:00:00.442971 4.86x         225.75/s
Inline $All.Where{}             00:00:00.371774 4.08x         268.98/s
2Step $All.Where{}              00:00:00.345280 3.79x         289.62/s
Gcm -CommandType Alias          00:00:00.091208 1x            1096.38/s



In [ ]:
$PSVersionTable.PSVersion | Join-String -op 'Pwsh: '
$Config = @{ Names = 'dev.nin' } 
h1 'Compare GCM itself, without filtering'
Measure-Benchmark -Technique @{
    # 'Filtering by the Source Command' = {
    'Gcm -m x -CommandType Alias' = {
        Get-Command -m $Config.Names -CommandType Alias
    }
    'Gcm -m' = {
        Get-Command -m $Config.Names
    }
    'Gcm * (implicit)' = {
        Get-Command
    }
    'Gcm * (explicit)' = {
        Get-Command *
    }
} | sort Time -desc

Pwsh: 7.2.5

# Compare GCM itself, without filtering




   Benchmark:  @~3696 Mhz

Technique                   Time            RelativeSpeed Throughput
---------                   ----            ------------- ----------
Gcm * (explicit)            00:00:47.655236 456.64x       2.1/s
Gcm * (implicit)            00:00:29.668240 284.28x       3.37/s
Gcm -m                      00:00:00.154693 1.48x         646.44/s
Gcm -m x -CommandType Alias 00:00:00.104361 1x            958.21/s

